# tf.Dataset, batch를 활용한 loader 만들기

In [1]:
import tensorflow as tf

# 일일 판매 데이터를 리스트로 정의합니다.
daily_sales_numbers = [21, 22, -100, 31, -1, 32, 34, 31]

# from_tensor_slices 메서드는 리스트나 배열 같은 데이터를 받아 Dataset 객체를 생성합니다.
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [2]:
# Dataset 객체의 각 요소를 순회하며 출력합니다.
for sales in tf_dataset:
    print(sales)

tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor(22, shape=(), dtype=int32)
tf.Tensor(-100, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)
tf.Tensor(-1, shape=(), dtype=int32)
tf.Tensor(32, shape=(), dtype=int32)
tf.Tensor(34, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)


### as_numpy_iterator

In [4]:
# as_numpy_iterator() 메서드를 사용하여 Dataset의 요소를 numpy 배열 형태로 변환하여 순회합니다.
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-100
31
-1
32
34
31


### take

In [5]:
# take() 메서드를 사용하여 Dataset에서 지정된 개수의 element를 가져옵니다.
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-100


### filter

In [6]:
# TensorFlow Dataset 객체를 필터링합니다.
# filter() 메서드를 사용하여 데이터셋에서 0보다 큰 요소만을 선택합니다.
tf_dataset = tf_dataset.filter(lambda x: x > 0)

for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


### map

- Dataset 전체에 함수를 맵핑합니다.

In [7]:
# map() 메서드를 사용하여 데이터셋의 각 요소를 2배로 증가시킵니다.
tf_dataset = tf_dataset.map(lambda x: x * 2)

for sales in tf_dataset.as_numpy_iterator():
    print(sales)

42
44
62
64
68
62


### shuffle
- 데이터세트는 buffer_size 요소로 버퍼를 채운 다음이 버퍼에서 요소를 무작위로 샘플링하여 선택한 요소를 새 요소로 바꿉니다.

- 완벽한 셔플 링을 위해서는 데이터 세트의 전체 크기보다 크거나 같은 버퍼 크기가 필요합니다.

- 예를 들어, 데이터 집합에 10,000 개의 element가 있지만 buffer_size가 1,000으로 설정된 경우 셔플은 처음에 버퍼의 처음 1,000 개 element 중 임의의 element 만 선택합니다.

- element가 선택되면 버퍼의 공간이 다음 element (즉, 1,001-st)로 대체되어 1,000 element 버퍼를 유지합니다.

In [8]:
# shuffle() 메서드를 사용하여 데이터셋의 순서를 무작위로 섞어 데이터의 순서에 의한 편향을 방지합니다.
tf_dataset = tf_dataset.shuffle(10)

for sales in tf_dataset.as_numpy_iterator():
    print(sales)

68
62
42
62
64
44


In [9]:
# batch() 메서드를 사용하여 각 배치의 크기를 2로 설정합니다.
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[44 42]
[62 64]
[62 68]


In [10]:
# tf_dataset의 데이터를 배치 크기 3으로 묶어서 출력
for sales_batch in tf_dataset.batch(3):
    print(sales_batch.numpy())

[62 42 62]
[68 44 64]


In [11]:
# tf_dataset의 데이터를 배치 크기 4으로 묶어서 출력
for sales in tf_dataset.batch(4):
    print(sales.numpy())

[62 62 42 64]
[44 68]


In [12]:
# tf_dataset의 데이터를 배치 크기 4로 묶어서 출력
# drop_remainder=True로 설정하여 마지막 배치가 배치 크기보다 작으면 해당 배치를 버림
for sales in tf_dataset.batch(4, drop_remainder=True):
    print(sales.numpy())

[62 62 68 64]


- 위 모든 것을 한 줄로 결합

In [13]:
# 일일 판매 데이터를 리스트로 정의합니다.
daily_sales_numbers = [21, 22, -100, 31, -1, 32, 34, 31]

# TensorFlow의 Dataset 객체를 생성합니다.
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

# 데이터셋을 필터링하여 0보다 큰 값만 유지합니다.
# 필터링된 데이터의 각 요소에 함수를 적용하여 값을 2배로 합니다.
# 데이터셋의 순서를 무작위로 섞습니다. 여기서는 버퍼 크기를 3으로 설정합니다.
# 데이터셋을 배치로 나눕니다. 각 배치는 4개의 요소를 포함하며, 마지막에 남는 요소는 버립니다.
tf_dataset = tf_dataset.filter(lambda x: x > 0).map(lambda y: y * 2).shuffle(3).batch(4, drop_remainder=True)

for sales_batch in tf_dataset.as_numpy_iterator():
    print(sales_batch)

[62 64 68 62]


### train, test dataset 생성
- train : test = 8 : 2

In [14]:
# 연속된 숫자(0부터 99까지)로 이루어진 리스트를 생성합니다.
daily_sales_numbers = list(range(100))

# from_tensor_slices 메서드를 사용하여 Dataset 객체를 생성합니다.
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

# 생성된 Dataset의 전체 크기를 계산합니다.
ds_count = len(tf_dataset)

# 전체 데이터셋의 80%를 훈련 데이터셋 크기로 설정합니다.
train_size = int(ds_count * 0.8)

# 처음부터 train_size 만큼의 데이터를 훈련 데이터셋으로 가져옵니다.
train_ds = tf_dataset.take(train_size)

# train_size 이후의 데이터를 테스트 데이터셋으로 설정합니다.
test_ds = tf_dataset.skip(train_size)

# 훈련 데이터셋과 테스트 데이터셋의 길이를 출력하여 확인합니다.
len(train_ds), len(test_ds)

(80, 20)

## StringLookup
- 문자열을 정수 index로 mapping 하여 vocabulary 사전 작성

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import StringLookup

 # 어휘 사전으로 사용될 리스트
vocab = ["apple", "banana", "orange", "grape"]

# 어휘 사전을 이용해 정수로 변환할 예제 데이터
data = [["apple", "orange", "banana"],
        ["banana", "grape", "apple"]]

# StringLookup 레이어를 생성하고 어휘(vocabulary)를 설정합니다. mask_token=None은 마스킹을 사용하지 않겠다는 의미입니다.
string_lookup = StringLookup(vocabulary=vocab, mask_token=None)
string_lookup

<StringLookup name=string_lookup, built=False>

In [16]:
# StringLookup 레이어를 사용하여 데이터 리스트에서 각 문자열을 해당하는 정수 인덱스로 변환합니다.
indices = string_lookup(data)

# 변환된 정수 인덱스를 출력합니다.
print(indices)

# StringLookup 레이어에 설정된 어휘 사전을 조회합니다.
vocab_list = string_lookup.get_vocabulary()

# 조회한 어휘 사전을 출력합니다.
print("Vocabulary:", vocab_list)

tf.Tensor(
[[1 3 2]
 [2 4 1]], shape=(2, 3), dtype=int64)
Vocabulary: ['[UNK]', 'apple', 'banana', 'orange', 'grape']


## Embedding Layer

In [17]:
from tensorflow.keras.layers import Embedding

# Embedding 레이어 설정
# 여기서는 최대 어휘 크기를 10000으로, 임베딩 벡터의 차원을 32로 설정합니다.
embedding_layer = Embedding(input_dim=10000, output_dim=32)

# 간단한 모델 생성 및 Embedding 레이어 추가
model = tf.keras.Sequential([
    embedding_layer
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [21]:
# 예제 데이터 - 일반적으로 여기서는 각 정수가 어휘 사전의 단어를 나타냅니다.
# 예를 들어, 5는 어휘 사전의 6번째 단어를 의미합니다(인덱싱은 0부터 시작).
input_data = np.array([[1, 2, 3, 4, 5],
                                    [5, 4, 3, 2, 1]])

# 모델을 통해 입력 데이터를 임베딩 벡터로 변환
output_data = model.predict(input_data)

# 임베딩 결과 출력
print(output_data.shape)
# print(output_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
(2, 5, 32)


In [22]:
# 어휘 사전을 생성합니다. 이는 나중에 텍스트 데이터를 숫자로 매핑하는 데 사용됩니다.
vocab = ['apple', 'banana', 'orange', 'grape']
vocab_size = len(vocab)  # 어휘 사전의 크기를 계산합니다.

# 예제 텍스트 데이터를 정의합니다.
texts = ['apple', 'banana', 'orange', 'grape']

# StringLookup 레이어를 생성합니다. 이 레이어는 텍스트를 정수 인덱스로 변환합니다.
# mask_token=None은 마스크 토큰을 사용하지 않겠다는 의미이고, output_mode='int'는 출력을 정수 인덱스로 설정합니다.
string_lookup = StringLookup(vocabulary=vocab, mask_token=None, output_mode='int')

# Embedding 레이어를 설정합니다.
# input_dim은 입력 차원으로 어휘 사전의 크기에 1을 더한 값입니다. 이 1은 StringLookup 레이어가 사용하는 패딩 인덱스 0을 위한 것입니다.
# output_dim은 출력 차원으로, 여기서는 임베딩 벡터의 크기를 8로 설정했습니다.
embedding_layer = Embedding(input_dim=vocab_size + 1, output_dim=8)

# Sequential 모델을 생성하고, 위에서 정의한 StringLookup 레이어와 Embedding 레이어를 추가합니다.
model = tf.keras.Sequential([
    string_lookup,
    embedding_layer
])

# 텍스트 데이터를 상수 텐서로 변환합니다.
input_data = tf.constant(texts)
# 모델을 사용하여 텍스트 데이터를 정수 인덱스로 변환한 후 해당 인덱스에 대응하는 임베딩 벡터를 얻습니다.
embedded_data = model(input_data)

# 모델 구조를 요약하여 출력합니다.
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string_lookup_1 (StringLookup)       │ (4)                         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_1 (Embedding)              │ (4, 8)                      │              40 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 40 (160.00 B)

 Trainable params: 40 (160.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
# 임베딩된 데이터의 형태를 출력
print("Embedded data:", embedded_data.numpy().shape)
# 임베딩된 데이터를 넘파이 배열로 변환하여 출력
embedded_data.numpy()

Embedded data: (4, 8)


array([[ 0.02339094, -0.04399924,  0.00475561,  0.00702316,  0.02135712,
         0.03861085, -0.02578144, -0.01207811],
       [-0.00494865, -0.04915276, -0.04207743, -0.04613136,  0.03060498,
         0.03861891,  0.04331971,  0.00176092],
       [-0.03572936, -0.02243661,  0.03338191, -0.0366606 ,  0.00078827,
        -0.0184085 ,  0.0030928 , -0.00068544],
       [ 0.01469838,  0.00903266,  0.02276088, -0.00787131, -0.01647718,
        -0.02739985, -0.03561059, -0.03342037]], dtype=float32)